In [1]:
from flask import Flask, request, jsonify
import json
from uuid import uuid4
import hashlib # hash 함수용 sha256 사용할 라이브러리
import json
from time import time
# from urlparse import urlparse
from urllib.parse import urlparse
import requests

from Blockchain import Blockchain
import datetime
app = Flask(__name__)
# Universial Unique Identifier
# 노드 식별을 하기 위해 uuid 함수를 사용!
node_identifier = 'node_5001' #  hashlib.sha256('0.0.0.0:5001'.encode()).hexdigest()
# 아까 짜놓은 블록체인 객체를 선언
blockchain = Blockchain()
my_ip = '0.0.0.0'
my_port = '5001'

node_identifier = 'node_' + my_port ##hashlib.sha256('0.0.0.0:5000'.encode()).hexdigest()

mine_owner = 'master'
mine_profit = 0.1

In [2]:

## chain 정보 얻기
@app.route('/chain', methods=['GET'])
def full_chain():
    print("chain info requested!!")
    response = {
        'chain' : blockchain.chain, # 블록체인을 출력
        'length' : len(blockchain.chain), # 블록체인 길이 출력
    }

    # json 형태로 리턴 (200 은 웹 사이트 에러가 없을 때 뜨는 숫자)
    return jsonify(response), 200

# post는 url에 데이터를 붙여서 보내는 get과 달리 숨겨서 보내는 방식
@app.route('/transactions/new', methods=['POST'])
def new_transaction():
    values = request.get_json() # json 형태를 받아서 저장
    print("transactions_new!!! : ", values)
    required = ['sender', 'recipient', 'amount'] # 해당 데이터가 존재해야함
    # 데이터가 없으면 에러를 띄움
    if not all(k in values for k in required):
        return 'missing values', 400

    # Create a new Transaction
    # 새 트랜잭션 만들고 삽입
    index = blockchain.new_transaction(values['sender'],values['recipient'],values['amount'])
    response = {'message' : 'Transaction will be added to Block {%s}' % index}
    
    ###################
    ## 내 노드에 있는 애들을 모두 다같이 업데이트 해줘야!!??
    
    if "type" in values:
        ## 다른노드가 전파하려고 준건 업데이트 필요 없고 나머지만 하면됨!!!!
        for node in blockchain.nodes:

            headers = {'Content-Type' : 'application/json; charset=utf-8'}
            data = {
                "sender": values['sender'],
                "recipient": values['recipient'],
                "amount": values['amount'],
                "type" : "sharing"
            }
            requests.post("http://" + node + "/transactions/new", headers=headers, data=json.dumps(data))
            print("share transaction to >>   ","http://" + node )
    return jsonify(response), 201

# 채굴이 되게 할 것
# coinbase transaction: 채굴할 때마다 1 코인씩 준다
@app.route('/mine', methods=['GET'])
def mine():
    
    last_block = blockchain.last_block
    last_proof = last_block['proof']

    proof = blockchain.pow(last_proof)  ## 여기가 진정한 채굴단계!!!! pow 에 while True가있어서 오래걸림!!

    blockchain.new_transaction(
        sender=mine_owner, # 채굴시 생성되는 transaction (0 = 운영자)
        recipient=node_identifier, # 지갑 주소처럼 사용
        amount=mine_profit # coinbase transaction 코인 1개를 줄게!!
    )
        ####################
    ## 내 노드에 있는 애들을 모두 다같이 업데이트 해줘야!!??
    for node in blockchain.nodes:
                
        headers = {'Content-Type' : 'application/json; charset=utf-8'}
        data = {
            "sender":  mine_owner,
            "recipient": node_identifier,
            "amount": mine_profit
        }
        requests.post("http://" + node + "/transactions/new", headers=headers, data=json.dumps(data))
  
    # Forge the new Block by adding it to the chain
    # 전 블록에 대한 hash를 떠놓고
    previous_hash = blockchain.hash(last_block)
    # 검증하는 걸 넣어서 블록을 새로 생성 !
    print("MINING STARTED")
    block = blockchain.new_block(proof, previous_hash)
    print("MINING FINISHED")

    ## 내가 채굴되었으면 남들한태는 새로운 블록정보를 업데이트해줘야해!!!!!
    ## 그렇게 검증도 받아야하고
    ################
    for node in blockchain.nodes:
        headers = {'Content-Type' : 'application/json; charset=utf-8'}
        data = {
            "miner_node":  'http://' + my_ip + ":" + my_port,
        }
            
        a = requests.get("http://" + node + "/nodes/resolve", headers=headers, data =json.dumps(data) ) 
#     print(a.text)
    # 이상이 없으면 정상배출
    if "ERROR" not in a.text :
        print("다른노드가 내블록 검증, 결과 정상!!!!!!")
        # block 이 제대로 mine 되었다는 정보를 json 형태로 띄워줌
        response = {
            'message' : 'new block found',
            'index' : block['index'],
            'transactions' : block['transactions'],
            'proof' : block['proof'],
            'previous_hash' : block['previous_hash']
        }
        
    #이상이 있다면!?!?
    else:
        1==1
        print("다른노드가 내블록 검증, 에러발생!!!!!!")
        #문제가 있음을 알려주자!!
        ## 과거데이터에 조작이있던것으로 내 블록 이 다시 타인의 블록정보 받아와야해!
    

    return jsonify(response), 200


## 나도 채굴할래!! 노드 등록하기!!
@app.route('/nodes/register', methods=['POST'])
def register_nodes():
    values = request.get_json() # json 형태로 보내면 노드가 저장이 됨
    print("register nodes !!! : ", values)
    nodes = [ values.get('nodes')]
    if nodes is None: # Bad Request 400
        return "Error: Please supply a valid list of nodes", 400

    for node in nodes:
        ## 중복인거면 스킵
        if node.split("//")[1] in blockchain.nodes:
            print("Node already registered")
            continue
        
        ## 중복 안댔으면 나에게 등록
        else:  
            blockchain.register_node(node) 

            ## 그리고 등록된 애들한테 
            ## 1. 내정보를  등록하기
            headers = {'Content-Type' : 'application/json; charset=utf-8'}
            data = {
                "nodes": 'http://' + my_ip + ":" + my_port
            }
            print("MY NODE INFO " , 'http://' + my_ip + ":" + my_port)
            requests.post( node + "/nodes/register", headers=headers, data=json.dumps(data))


    response = {
        'message' : 'New nodes have been added',
        'total_nodes' : list(blockchain.nodes),
    }
    return jsonify(response), 201


## 새로온 친구야 방가워! 다른 노드 정보들을 줄게!!
@app.route('/nodes/check', methods=['GET'])
def check_nodes():
    now_ = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    response = {
        'message' : 'Node List at ' + now_,
        'total_nodes' : list(blockchain.nodes),
    }
    return jsonify(response), 201

## 내가 해결했어!@!! 그내용을 전체에 공유할게!!
@app.route('/nodes/resolve', methods=['GET'])
def consensus():
    
    requester_node_info =  request.get_json()
    required = ['miner_node'] # 해당 데이터가 존재해야함
    # 데이터가 없으면 에러를 띄움
    if not all(k in requester_node_info for k in required):
        return 'missing values', 400   
    
    
    
    ## 그전에 우선 previous 에서 바뀐것이 있는지 점검하자!!
    my_previous_hash =  blockchain.last_block['previous_hash']
    my_previous_hash
    
    headers = {'Content-Type' : 'application/json; charset=utf-8'}
    miner_chain_info = requests.get(requester_node_info['miner_node'] + "/chain", headers=headers)
    ##초기 블록은 과거 이력 변조내역 확인 할 필요가 없다
        
    print("다른노드에서 요청이 온 블록, 검증 시작")
    new_block_previous_hash = json.loads(miner_chain_info.text)['chain'][-2]['previous_hash']
    # 내꺼의 전해시랑 새로만든에의 전해시가 같을떄!!! >> 정상
    if my_previous_hash == new_block_previous_hash:
        print("다른노드에서 요청이 온 블록, 검증결과 정상!!!!!!")

        replaced = blockchain.resolve_conflicts() # True Flase return

        # 체인 변경 알림 메시지
        if replaced:
            ## 내 체인이 짧아서 권위가 없음
            print("REPLACED length :",len(blockchain.chain))
            response = {
                'message' : 'Our chain was replaced >> ' +  my_ip + ":"+ my_port,
                'new_chain' : blockchain.chain
            }
        else:
            ## 내 체인이 제일 길어서 권위가 있음
            response = {
                'message' : 'Our chain is authoritative',
                'chain' : blockchain.chain
            }
    #아니면 무엇인가 과거데이터가 바뀐것이다!!
    else:
        print("다른노드에서 요청이 온 블록, 검증결과 이상발생!!!!!!!!")
        response = {
            'message' : 'Our chain is authoritative>> ' +  my_ip + ":"+ my_port,
            'chain' : blockchain.chain
        }            


    return jsonify(response), 200

if __name__ == '__main__':
    app.run(host=my_ip, port=my_port) 

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://172.30.1.11:5001/ (Press CTRL+C to quit)
127.0.0.1 - - [19/May/2022 10:15:52] "POST /nodes/register HTTP/1.1" 201 -


register nodes !!! :  {'nodes': 'http://0.0.0.0:5000'}
MY NODE INFO  http://0.0.0.0:5001


127.0.0.1 - - [19/May/2022 10:15:54] "POST /transactions/new HTTP/1.1" 201 -
127.0.0.1 - - [19/May/2022 10:15:54] "GET /nodes/resolve HTTP/1.1" 200 -


transactions_new!!! :  {'sender': 'master', 'recipient': 'node_5000', 'amount': 0.1}
다른노드에서 요청이 온 블록, 검증 시작
다른노드에서 요청이 온 블록, 검증결과 정상!!!!!!
{'index': 1, 'previous_hash': 1, 'proof': 100, 'timestamp': 1652922951.8196206, 'transactions': []}
{'index': 2, 'previous_hash': 'b900abd84614cb9fbb28b2dc76d71361d799fb91cf9f0b578b7e34e79a12e1ff', 'proof': 88384, 'timestamp': 1652922954.1798725, 'transactions': [{'amount': 3, 'recipient': 'zzz', 'sender': 'abc', 'timestamp': 1652922953.1153772}, {'amount': 1003, 'recipient': 'yyy', 'sender': 'def', 'timestamp': 1652922953.121301}, {'amount': 143, 'recipient': 'ccc', 'sender': 'tyu', 'timestamp': 1652922953.1262085}, {'amount': 0.1, 'recipient': 'node_5000', 'sender': 'master', 'timestamp': 1652922954.1748877}]}

--------

REPLACED length : 2


...

transactions_new!!! :  {'sender': 'abc2', 'recipient': 'zzz2', 'amount': 3}
transactions_new!!! :  {'sender': 'zzz2', 'recipient': 'def2', 'amount': 3}
transactions_new!!! :  {'sender': 'def2', 'recipient': 'yyy2', 'amount': 1003}
transactions_new!!! :  {'sender': 'tyu2', 'recipient': 'ccc2', 'amount': 143}


127.0.0.1 - - [19/May/2022 10:22:47] "POST /transactions/new HTTP/1.1" 201 -
127.0.0.1 - - [19/May/2022 10:22:47] "POST /transactions/new HTTP/1.1" 201 -
127.0.0.1 - - [19/May/2022 10:22:47] "POST /transactions/new HTTP/1.1" 201 -
127.0.0.1 - - [19/May/2022 10:22:47] "POST /transactions/new HTTP/1.1" 201 -


transactions_new!!! :  {'sender': 'abc2', 'recipient': 'zzz2', 'amount': 3}
transactions_new!!! :  {'sender': 'zzz2', 'recipient': 'def2', 'amount': 3}
transactions_new!!! :  {'sender': 'def2', 'recipient': 'yyy2', 'amount': 1003}
transactions_new!!! :  {'sender': 'tyu2', 'recipient': 'ccc2', 'amount': 143}


127.0.0.1 - - [19/May/2022 10:22:49] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2022 10:22:49] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2022 10:22:49] "GET /mine HTTP/1.1" 200 -


MINING STARTED
MINING FINISHED
chain info requested!!
chain info requested!!
다른노드가 내블록 검증, 결과 정상!!!!!!


In [ ]:
 blockchain.last_block['previous_hash']

In [ ]:
last_block = blockchain.last_block
my_previous_hash = blockchain.hash(last_block)    
my_previous_hash

In [ ]:
 json.loads(a.text)['chain'][-2]['previous_hash']

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
a = requests.get("http://localhost:5000/chain", headers=headers)

In [ ]:
json.loads(a.text)['chain'][-2]['previous_hash']

In [ ]:
blockchain.chain

In [ ]:
json.loads(a.text)['chain'][-2]#['previous_hash']